# Logging, Tracking, and Debugging Prompts using Comet

In this section, we will demonstrate how to log, track, and debug prompt using the `comet-llm` library. `comet-llm` is an open-sourced repo managed by Comet. Please give the repo star if you have a chance and submit any feedback you have! https://github.com/comet-ml/comet-llm

Let's first load all the necessary libraries:


In [ ]:
! pip install openai comet-llm --quiet

In [ ]:
import openai
import os
import IPython
import json
import pandas as pd
import numpy as np
import comet_llm
import urllib


The function below helps to generate the final results from the model after calling the OpenAI API:

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

### Load the Data

The code below loads both the few-shot demonstrations and the validation dataset used for testing the model.

In [ ]:
# API configuration
openai.api_key = "OPENAI_API_KEY"

# print markdown
def print_markdown(text):
    """Prints text as markdown"""
    IPython.display.display(IPython.display.Markdown(text))

# load validation data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/article-tags.json")
val_data = json.load(f)

# load few shot data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/few_shot.json")
few_shot_data = json.load(f)

The following is a helper function to obtain the final predictions from the model given a prompt template (e.g., zero-shot or few-shot) and the provided input data.

In [ ]:
def get_predictions(prompt_template, inputs):

    responses = []

    for i in range(len(inputs)):
        messages = messages = [
            {
                "role": "system",
                "content": prompt_template.format(input=inputs[i])
            }
        ]
        response = get_completion(messages)
        responses.append(response)

    return responses

### Few-Shot

First, we define a few-shot template which will leverage the few-shot demonstration data loaded previously.

In [ ]:
# function to define the few-shot template
def get_few_shot_template(few_shot_prefix, few_shot_suffix, few_shot_examples):
    return few_shot_prefix + "\n\n" + "\n".join([ "Abstract: "+ ex["abstract"] + "\n" + "Tags: " + str(ex["tags"]) + "\n" for ex in few_shot_examples]) + "\n\n" + few_shot_suffix

# function to sample few shot data
def random_sample_data (data, n):
    return np.random.choice(few_shot_data, n, replace=False)


# the few-shot prefix and suffix
few_shot_prefix = """Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]"""
few_shot_suffix = """Abstract: {input}\nTags:"""

# load 3 samples from few shot data
few_shot_template = get_few_shot_template(few_shot_prefix, few_shot_suffix, random_sample_data(few_shot_data, 3))

In [ ]:
few_shot_template

'Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format ["model_name"]. If you don\'t find model names in the abstract or you are not sure, return ["NA"]\n\nAbstract: Large Language Models (LLMs), such as ChatGPT and GPT-4, have revolutionized natural language processing research and demonstrated potential in Artificial General Intelligence (AGI). However, the expensive training and deployment of LLMs present challenges to transparent and open academic research. To address these issues, this project open-sources the Chinese LLaMA and Alpaca large models, emphasizing instruction fine-tuning. We expand the original LLaMA\'s Chinese vocabulary by adding 20K Chinese tokens, increasing encoding efficiency and enhancing basic semantic understanding. By incorporating secondary pre-training using Chinese data and fine-tuning with Chinese instruction data, we substantially improve the models\' comprehension and ex

### Zero-Shot Template

The code below defines the zero-shot template. Note that we use the same instruction from the few-shot prompt template. But in this case, we don't use the demonstrations.

In [ ]:
zero_shot_template = """
Your task is extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]

Abstract: {input}
Tags:
"""

### Get Predictions

We then generated all the predictions using the validation data as inputs:

In [ ]:
# get the predictions

abstracts = [val_data[i]["abstract"] for i in range(len(val_data))]
few_shot_predictions = get_predictions(few_shot_template, abstracts)
zero_shot_predictions = get_predictions(zero_shot_template, abstracts)
expected_tags = [str(val_data[i]["tags"]) for i in range(len(val_data))]

In [ ]:
print("Few shot predictions")
print(few_shot_predictions)
print("\n\nZero shot predictions")
print(zero_shot_predictions)
print("\n\nExpected tags")
print(expected_tags)

Few shot predictions
["['LLM', 'ChatGPT', 'LLaMA', 'WizardLM', 'OpenAI ChatGPT']", "['FLAN-T5', 'AMR', 'UD', 'SRL', 'LoRA']", '["NA"]', "['ChatGPT', 'GPT-4', 'LLaMA', 'Alpaca', 'GMMSeg', 'GMMs', 'PAXQA']", "['ChatGPT']", "['ViT', 'OpenCLIP']", "['SAM', 'IA', 'AIGC', 'Stable Diffusion']", "['Anything-3D', 'BLIP', 'Segment-Anything']", "['Chameleon', 'LLMs', 'GPT-4', 'ScienceQA', 'TabMWP', 'ChatGPT']", "['NA']"]


Zero shot predictions
['["WizardLM", "Evol-Instruct", "LLaMA", "ChatGPT"]', '["FLAN-T5"]', '["large language models", "generative AI", "hypothesis machines"]', '["PAXQA"]', '["ChatGPT"]', '["ViT", "OpenCLIP"]', '["Segment-Anything Model (SAM)", "Inpaint Anything (IA)", "AIGC models", "Stable Diffusion", "Inpaint Anything (IA)"]', '["Anything-3D", "BLIP", "Segment-Anything", "text-to-image diffusion model"]', '["Chameleon", "GPT-4", "ScienceQA", "TabMWP", "ChatGPT"]', '["NA"]']


Expected tags
["['LLaMA', 'ChatGPT', 'WizardLM']", "['FLAN-T5', 'FLAN']", "['NA']", "['PAXQA']", "['

### Log Prompt Results

Finally, we log the prompt + results to Comet. Note that we are logging both the few-shot and zero-shot results, together with all the metadata and tags.

In [ ]:
# log the predictions in Comet along with the ground truth for comparison

# set up comet
COMET_API_KEY = "COMET_API_KEY"
COMET_WORKSPACE = "COMET_WORKSPACE"

# initialize comet
comet_llm.init(COMET_API_KEY, COMET_WORKSPACE, project="ml-paper-tagger-prompts")

# log the predictions
for i in range(len(expected_tags)):
    # log the few-shot predictions
    comet_llm.log_prompt(
        prompt=few_shot_template.format(input=abstracts[i]),
        prompt_template=few_shot_template,
        output=few_shot_predictions[i],
        tags = ["gpt-3.5-turbo", "few-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

    # log the zero-shot predictions
    comet_llm.log_prompt(
        prompt=zero_shot_template.format(input=abstracts[i]),
        prompt_template=zero_shot_template,
        output=zero_shot_predictions[i],
        tags = ["gpt-3.5-turbo", "zero-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

COMET INFO: Valid Comet API Key saved in /Users/elvissaravia/.comet.config (set COMET_CONFIG to change where it is saved).


Prompt logged to https://www.comet.com/omarsar/ml-paper-tagger-prompts
